In [ ]:
text = "This is a test document."

doc_result = gpt4all_embd.embed_documents([text])

doc_result

In [54]:
import chromadb
import pandas as pd
from langchain.embeddings import GPT4AllEmbeddings

gpt4all_embd = GPT4AllEmbeddings()

client = chromadb.PersistentClient(path="db/")

collection = client.create_collection(name="test")

# Assuming you have the necessary variables defined
item_to_be_saved = f"carlitos way thriller 5 Al Pacino Sean Penn Penelope Ann Miller John Leguizamo Brian De Palma Carlito Brigante is released from jail after serving five years of a much longer stretch thanks to a technicality and he vows to begin a new life without the influence of drugs As a way to",

# Insert item_to_be_saved into the embedding engine 
doc_result = gpt4all_embd.embed_documents([item_to_be_saved])
doc_result

# Save the embedded document to Chroma

# Use the following code as an example
# collection.add(
#     documents = [student_info, club_info, university_info],
#     metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
#     ids = ["id1", "id2", "id3"]
# )

KeyboardInterrupt: 

In [1]:
%pip install python-dotenv > /dev/null

/home/user/.local/lib/python3.10/site-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/user/.local/lib/python3.10/site-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.
